# Imports 🔧

In [1]:
import random
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from surprise import SVD, Dataset, Reader


# Preprocesamiento del dataset 🧹

In [2]:
def set_seed(seed=1):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    try:
        from surprise import SVD, Dataset, Reader
        np.random.seed(seed)
    except ImportError:
        pass

set_seed(1)


In [3]:
train_data = pd.read_csv("data/train.csv", sep=",")
test_data = pd.read_csv("data/test.csv", sep=",")

In [4]:
train_data.head(5)

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0


In [5]:
test_data.head(5)

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128


In [6]:
#train_data, val_split = train_test_split(train_data, test_size=0.2, random_state=42)

## Creación de matriz dispersa 🧩

In [7]:
unique_users = train_data['user'].unique()
unique_items = train_data['item'].unique()

user_to_index = {user: idx for idx, user in enumerate(unique_users)}
item_to_index = {item: idx for idx, item in enumerate(unique_items)}

num_users = len(unique_users)
num_items = len(unique_items)

global_mean = train_data['rating'].mean()

print(f"Número de usuarios: {num_users}")
print(f"Número de ítems: {num_items}")
print(f"Media global de ratings: {global_mean:.4f}")

Número de usuarios: 73456
Número de ítems: 171171
Media global de ratings: 7.6047


In [ ]:
# Crea la matriz dispersa
rows = train_data['user'].map(user_to_index).values
cols = train_data['item'].map(item_to_index).values
ratings = train_data['rating'].values

R_sparse = csr_matrix((ratings, (rows, cols)), shape=(num_users, num_items))

print(f"Matriz dispersa creada con dimensiones: {R_sparse.shape}")
print("Número de ratings no nulos en la matriz:", R_sparse.nnz)

Matriz dispersa creada con dimensiones: (73456, 171171)
Número de ratings no nulos en la matriz: 390351


# Entrenamiento PMF 🏋️

In [9]:
set_seed(1)

# Hiperparámetros
k = 20  # Número de factores latentes
lambda_ = 0.01  # Regularización
learning_rate = 0.0025  # Tasa de aprendizaje
num_epochs = 50  # Número de iteraciones

mu = global_mean

# Inicialización aleatoria de las matrices latentes U y V
U = np.random.normal(scale=1./k, size=(num_users, k))
V = np.random.normal(scale=1./k, size=(num_items, k))

# Inicialización de sesgos
bu = np.zeros(num_users)
bi = np.zeros(num_items)

print(f"Factores latentes inicializados: U ({U.shape}), V ({V.shape})")
print(f"Sesgos inicializados: bu ({bu.shape}), bi ({bi.shape})")


Factores latentes inicializados: U ((73456, 20)), V ((171171, 20))
Sesgos inicializados: bu ((73456,)), bi ((171171,))


In [10]:
def train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, lr, num_epochs):
    rows, cols = R_sparse.nonzero()
    num_ratings = len(rows)

    for epoch in range(num_epochs):
        total_cost = 0
        for idx in range(num_ratings):
            i = rows[idx]  # Usuario
            j = cols[idx]  # Ítem
            r_ij = R_sparse[i, j]

            # Predicción
            pred_ij = mu + bu[i] + bi[j] + np.dot(U[i, :], V[j, :])
            error = r_ij - pred_ij

            # Actualización de los factores latentes y sesgos
            bu[i] += lr * (error - lambda_ * bu[i])
            bi[j] += lr * (error - lambda_ * bi[j])
            U[i, :] += lr * (error * V[j, :] - lambda_ * U[i, :])
            V[j, :] += lr * (error * U[i, :] - lambda_ * V[j, :])

            # Coste regularizado
            total_cost += error**2 + (lambda_ / 2) * (np.linalg.norm(U[i, :])**2 + np.linalg.norm(V[j, :])**2 + bu[i]**2 + bi[j]**2)

        print(f"Epoch {epoch + 1}/{num_epochs} - Coste total: {total_cost:.4f}")

    return U, V, bu, bi

In [11]:
print("Iniciando entrenamiento PMF...")
U, V, bu, bi = train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, learning_rate, num_epochs)
print("Entrenamiento completado.")


Iniciando entrenamiento PMF...
Epoch 1/50 - Coste total: 1252112.2893
Epoch 2/50 - Coste total: 1179491.7264
Epoch 3/50 - Coste total: 1139154.8887
Epoch 4/50 - Coste total: 1109724.2523
Epoch 5/50 - Coste total: 1086067.9612
Epoch 6/50 - Coste total: 1066034.1274
Epoch 7/50 - Coste total: 1048487.5476
Epoch 8/50 - Coste total: 1032742.5712
Epoch 9/50 - Coste total: 1018338.3200
Epoch 10/50 - Coste total: 1004929.4995
Epoch 11/50 - Coste total: 992224.7303
Epoch 12/50 - Coste total: 979950.2713
Epoch 13/50 - Coste total: 967839.5536
Epoch 14/50 - Coste total: 955666.7858
Epoch 15/50 - Coste total: 943344.7203
Epoch 16/50 - Coste total: 931048.7257
Epoch 17/50 - Coste total: 919208.7601
Epoch 18/50 - Coste total: 908241.4476
Epoch 19/50 - Coste total: 898248.6917
Epoch 20/50 - Coste total: 889032.9117
Epoch 21/50 - Coste total: 880324.4149
Epoch 22/50 - Coste total: 871919.6835
Epoch 23/50 - Coste total: 863690.8177
Epoch 24/50 - Coste total: 855557.5122
Epoch 25/50 - Coste total: 84746

# Entrenamiento SVD 🏋️

In [13]:
# Fijar el seed para el modelo SVD
set_seed(1)

# Crear el modelo SVD
svd = SVD(n_factors=20, n_epochs=50, biased=True, lr_all=0.002, reg_all=0.02, random_state=42)

reader = Reader(rating_scale=(1, 10))
trainset = Dataset.load_from_df(train_data[['user', 'item', 'rating']], reader).build_full_trainset()

svd.fit(trainset)


# Predicciones y generación de CSV 🧠

In [14]:
# Predicción combinada ponderada con PMF y SVD
def predict_rating(uid, iid, U, V, bu, bi, mu):
    try:
        # Predicción usando SVD
        pred_svd = svd.predict(uid, iid).est
    except:
        pred_svd = None

    # Predicción usando PMF con bias
    if uid in user_to_index and iid in item_to_index:
        user_idx = user_to_index[uid]
        item_idx = item_to_index[iid]
        pred_pmf = mu + bu[user_idx] + bi[item_idx] + np.dot(U[user_idx, :], V[item_idx, :])
    else:
        pred_pmf = None

    # Ponderación de las predicciones (40% SVD y 60% PMF)
    if pred_svd is not None and pred_pmf is not None:
        pred = (0.4 * pred_svd) + (0.6 * pred_pmf)
    elif pred_svd is not None:
        pred = pred_svd
    elif pred_pmf is not None:
        pred = pred_pmf
    else:
        pred = mu

    return f"{round(max(1, min(10, pred)))}.0"

In [15]:
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid, iid, U, V, bu, bi, mu)
    predictions.append((row_id, pred_rating))

predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])

output_filename = "predictions_pmf_svd_2.csv"
predictions_df.to_csv(output_filename, index=False)
print(f"Archivo '{output_filename}' generado.")

Archivo 'predictions_pmf_svd_2.csv' generado.
